# Title: Imbalanced_Data_Weighted_Sampler
## Description: 데이터 클래스 비율의 역수(Inverse Frequency)를 가중치로 계산하여, DataLoader가 학습 시 소수 클래스를 더 자주 샘플링하도록(Over-sampling 효과) 만드는 샘플러 생성기.
## Input: 
 - targets (List or np.array): 전체 학습 데이터의 타겟(라벨) 리스트
## Output: 
 - sampler (WeightedRandomSampler): DataLoader에 주입 가능한 PyTorch 샘플러 객체
## Check Point: 
 - `DataLoader` 사용 시 `shuffle=True` 옵션과 충돌하므로, 반드시 `shuffle=False`로 설정해야 함 (Sampler가 셔플 역할을 대신함).
 - 복원 추출!

In [ ]:
import torch
import numpy as np
from collections import Counter
from torch.utils.data import WeightedRandomSampler, DataLoader

def create_weighted_sampler(targets):
    """
    클래스 불균형 해결을 위한 WeightedRandomSampler 생성 함수
    """
    # 1. 각 클래스별 개수 산출
    class_counts = Counter(targets)
    
    # 2. 클래스별 가중치 계산 (개수의 역수)
    class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
    
    # 3. 각 샘플(데이터 한 줄 한 줄)마다 가중치 부여
    sample_weights = [class_weights[t] for t in targets]
    
    # 4. 샘플러 생성
    # weights: 각 샘플이 뽑힐 확률 (가중치)
    # num_samples: 한 에폭당 뽑을 데이터 수 (보통 전체 데이터 수와 동일하게 설정)
    # replacement=True: 복원 추출 (중요: 적은 데이터가 여러 번 뽑혀야 하므로 필수)
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    return sampler

## How to Use
1. **타겟 리스트 추출**: 학습 데이터프레임이나 데이터셋에서 라벨(y) 값만 리스트로 추출합니다.
2. **샘플러 생성 및 적용**:
    ```python
    # train_y는 정수형 인코딩이 완료된 라벨 리스트여야 합니다.
    sampler = create_weighted_sampler(train_dataset.labels)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=32, 
        sampler=sampler,  # <--- 여기서 주입
        shuffle=False,    # [주의] Sampler 사용 시 무조건 False!
        num_workers=4
    )
    ```

## Troubleshooting
- **에러 발생: `sampler option is mutually exclusive with shuffle`**: `DataLoader`에서 `sampler`를 넣고 `shuffle=True`를 켜서 발생하는 에러입니다. `shuffle=False`로 변경하세요.
- **과적합(Overfitting)**: 소수 클래스 데이터를 너무 많이 복제(복원 추출)하게 되므로, 소수 클래스에 대해 과적합이 발생할 수 있습니다. 이 경우 `Augmentation` 강도를 높여 똑같은 데이터가 들어가지 않도록 변형을 많이 주어야 합니다.